In [1]:
import json
import time
import csv
import re
import datetime
from enum import Enum, unique


import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [2]:
G_Debug = True   # if debug mode.  Debug = True. if Release mode, Debug = False.
G_Chrome_Hide = True
G_Debug_Pages = 2
G_Debug_Items = 5

G_FILE_NAME = 'tianjin'

In [3]:
#will bid
# sourceUrl = ("https://sf.taobao.com/item_list.htm?spm=a213w.7398504.filter.60.53db26cdCGMLqt&category=50025969"  
#                    "&auction_source=0&province=%CC%EC%BD%F2&sorder=1&st_param=-1&auction_start_seg=-1")
# #done bid
sourceUrl = ("https://sf.taobao.com/item_list.htm?spm=a213w.7398504.filter.26.294826cdrtlKJJ&category=50025969&auction_source=0"
             "&city=&province=%CC%EC%BD%F2&sorder=2&st_param=-1&auction_start_seg=-1")

In [4]:
def log(page_id,item_id,status,start):
    end = datetime.datetime.now()
    print("page id: {0}, item id: {1}, status:{2}, duration:{3}".format(page_id,item_id,status,end-start))

In [5]:
@unique
class Data(Enum):

    #release
    name = "中标人"
    year = "年份"
    month = "月份"
    start_time = "拍卖开始"
    end_time = "拍卖结束"
    times = "拍卖次数"
    title = "房屋地址"
    district = "区域"
    loadStatus = "贷款"
    originPrice = "市场评估价(万)"
    currentPrice = "当前价(万)"
    area = "建筑面积(平方米)"
    discount = "折扣(当前价/市场评估价)"
    unit = "单价(万)"
    bidCount = "举牌次数"
    delayCount = "举牌延迟次数"
    applyCount = "报名人数"    
    itemUrl = "网页链接"
    
    #debug
    areaA = "候选面积1(平方米)"
    areaB = "后面面积2(平方米)"
    id = "拍品编号"
    status = "拍卖状态"
    consultPrice = "评估价(元)"
    marketPrice = "市场价(元)"
    supportLoans = "普通贷款"
    supportOrgLoan = "法服贷款"
    start = "拍卖开始时间戳"
    end = "拍卖结束时间戳"

In [6]:
class DataManagement:
    def __init__(self,file_name):
        self.data = {}

        self.file_name = file_name
    
        self.head = []
        for data in Data:
            self.head.append(data.value)
        
        self.__open_file()

    def __process_attri_name(self,attribute, data):
        index = data.find("通过")
        data = data[4:index]
        self.data[attribute] =  data     
            
    def __process_attri_area(self,attribute, data):
        left = data.find("建筑面积")
        if(left != -1):
            size = data[(left+4) : (left+22)]
            areas = re.findall(r"\d+\.\d*",size)
            self.data[attribute] = areas[0] if areas else 0
        else:
            self.data[attribute] = 0
        

        
    def __process_attri_times(self,attribute, data):
        keyword_postion = data.find("卖")
        if(keyword_postion != -1):
            data = data[1:keyword_postion+1]
        else:
            data = "no content"
            
        self.data[attribute] = data



    def __process_attri_common(self,attribute, data):
        self.data[attribute] =  data        
     
    def __process_attri_url(self,attribute, data):
        url = "https:"+data
        self.data[attribute] =  url        

    
    def __clac_data(self):
        # 
        if (int(self.data[Data.consultPrice.name]) > 0):
            originPrice = self.data[Data.consultPrice.name]
        else:
            originPrice = self.data[Data.marketPrice.name]
        #
        if (int(originPrice)  == 0):
            self.data[Data.discount.name] = "error"
        else:
            self.data[Data.discount.name]  = float("%.2f" % float(int(self.data[Data.currentPrice.name])/originPrice))
 
       # change unit to from RMB yuan to  RMB Wan
        self.data[Data.originPrice.name]  = float("%.2f" % float(int(originPrice)/10000))        
        self.data[Data.currentPrice.name] = float("%.2f" % float(int(self.data[Data.currentPrice.name])/10000))

      # set area
        if(float(self.data[Data.areaA.name]) > 0):
            self.data[Data.area.name]  = self.data[Data.areaA.name]
            self.data[Data.unit.name] = float("%.2f" % float(int(self.data[Data.currentPrice.name])/float(self.data[Data.area.name]) ))
        elif(float(self.data[Data.areaB.name]) > 0):
            self.data[Data.area.name] = self.data[Data.areaB.name]
            self.data[Data.unit.name] = float("%.2f" % float(int(self.data[Data.currentPrice.name])/float(self.data[Data.area.name]) ))
        else:
            self.data[Data.area.name] = 0
            self.data[Data.unit.name] = 0
        
        ## start_time, end_time
        start = int(int(self.data[Data.start.name])/1000 )
        self.data[Data.start_time.name] = str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(start)))
        
        end = int(int(self.data[Data.end.name])/1000 )
        self.data[Data.end_time.name] = str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(end)))
        
        ## set year, month
        date_start = self.data[Data.start_time.name]
        data_split = date_start.split("-",2)
        self.data[Data.year.name] = data_split[0]
        self.data[Data.month.name] = data_split[1]
        ## set load
        if int(self.data[Data.supportLoans.name]) == 0 and int(self.data[Data.supportOrgLoan.name]) == 0:
            self.data[Data.loadStatus.name] = "不支持"
        elif int(self.data[Data.supportLoans.name]) == 1 and int(self.data[Data.supportOrgLoan.name]) == 1:
            self.data[Data.loadStatus.name] = "支持(法服&普通)"
        elif int(self.data[Data.supportOrgLoan.name]) == 1:
            self.data[Data.loadStatus.name] = "支持(法服)"
        else:
            self.data[Data.loadStatus.name] = "支持(普通)"

        ## district
        address = self.data[Data.title.name]
        city_pos = address.find("市")
        district_pos = address.find("区")

        if city_pos > 0 and district_pos > 0:
            self.data[Data.district.name] = address[city_pos+1:district_pos+1]
        elif city_pos == -1 and district_pos > 0:
            city_pos = address.find("天津")
            if city_pos > -1 :
                self.data[Data.district.name] = address[city_pos+2:district_pos+1]
            else:
                self.data[Data.district.name] = address[0:district_pos+1]
        else:
            self.data[Data.district.name] = "未知区"
        ###

        
    
    def set_data(self,attribute, data):
        if(attribute == Data.name.name):
            self.__process_attri_name(attribute, data)
        elif(attribute == Data.areaA.name or attribute == Data.areaB.name):
            self.__process_attri_area(attribute, data)
        elif(attribute == Data.times.name):
            self.__process_attri_times(attribute, data)
        elif(attribute == Data.itemUrl.name):
            self.__process_attri_url(attribute, data)
        else:
            self.__process_attri_common(attribute,data)    
    
    
    def get_data(self,attribute):
        return self.data[attribute]
    
    def __open_file(self):
        self.csv_file = open(self.file_name+".csv","w+")
        self.writer = csv.writer(self.csv_file)        
        self.writer.writerow(self.head)

    def write_file(self):
        
        self.__clac_data()
        
        row = tuple(self.data[d.name] for d in Data)
        
        self.writer.writerow(row)
    
    def close_file(self):
        self.csv_file.close()
        csv = pd.read_csv(self.file_name+".csv", encoding='utf-8')
        
        #debug excel
        csv.to_excel(self.file_name+"_debug.xlsx", sheet_name='data')
        
        #realse excel
        relase_column = list(self.head)
        relase_column.remove(Data.areaA.value)
        relase_column.remove(Data.areaB.value)
        relase_column.remove(Data.id.value)
        relase_column.remove(Data.status.value)
        relase_column.remove(Data.consultPrice.value)
        relase_column.remove(Data.marketPrice.value)
        relase_column.remove(Data.supportLoans.value)
        relase_column.remove(Data.supportOrgLoan.value)
        relase_column.remove(Data.start.value)
        relase_column.remove(Data.end.value)




        
        csv.to_excel(self.file_name+"_release.xlsx", sheet_name='data',columns =relase_column )
            


In [7]:
class SfSpider:
    
    
    def __init__(self,debug,debug_pages,debug_items,head_less,url,file_name):
        
        self.debug = debug
        self.debug_pages = debug_pages
        self.debug_items = debug_items
        self.url = url

        
        
        self.data = DataManagement(file_name)
        
        if(head_less):
            #set headless
            chrome_options=Options()
            chrome_options.add_argument('--headless')
            self.driver = webdriver.Chrome(options=chrome_options)
        else:
            self.driver = webdriver.Chrome()
            

        # Setup wait for later
        self.wait = WebDriverWait(self.driver, 10)

        
    def __wait_and_click(self,mode,value,sleep_time):
    #  Sometimes click fails unreasonably. So tries to click at all cost.
        try:
            if(mode == "LINK_TEXT"):
                elem = self.wait.until(EC.element_to_be_clickable((By.LINK_TEXT, value)))
            elif(mode == "CSS_SELECTOR"):
                elem = self.wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,value)))
            elif(mode == "XPATH"):
                elem = self.wait.until(EC.element_to_be_clickable((By.XPATH,value)))
            else:
                print('wait_and_click() error...')
                
            elem.click()
            time.sleep(sleep_time)
        except Exception as e:
            print('Click time out - {}'.format(value))
            print('Refreshing browser...')
            self.driver.refresh()
            time.sleep(2)
            return self.__wait_and_click(mode,value,sleep_time)

        return elem
        
        
    def do_crawling(self):
        
        self.driver.get(self.url)
        
        assert len(self.driver.window_handles) == 1

        elem = self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))
        
        if(self.debug):
            pages = self.debug_pages
        else:
            pages = int(self.driver.find_element_by_class_name("page-total").text)
                    
        for page in range(1,pages+1):
            self.__do_page_crawling(page)

        self.driver.quit()
        self.data.close_file()
        
        

    def __do_page_crawling(self,page):
        if(page > 1):
            self.__wait_and_click("LINK_TEXT",str(page),1)

        self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sf-pai-item-list")))
        jsonData =  json.loads(self.driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML'))
        page_data = jsonData["data"]

        if(self.debug):
            items = self.debug_items
        else:
            items = len(page_data)    

        for item in range(items):
            item_data = page_data[item]
            
            start = datetime.datetime.now()  
            
            self.data.set_data(Data.id.name,item_data[Data.id.name])
            self.data.set_data(Data.status.name,item_data[Data.status.name])
            self.data.set_data(Data.start.name,item_data[Data.start.name])
            self.data.set_data(Data.end.name,item_data[Data.end.name])
            self.data.set_data(Data.title.name,item_data[Data.title.name])
            self.data.set_data(Data.consultPrice.name,item_data[Data.consultPrice.name])
            self.data.set_data(Data.marketPrice.name,item_data[Data.marketPrice.name])
            self.data.set_data(Data.currentPrice.name,item_data[Data.currentPrice.name])            
            self.data.set_data(Data.bidCount.name,item_data[Data.bidCount.name])
            self.data.set_data(Data.delayCount.name,item_data[Data.delayCount.name])
            self.data.set_data(Data.applyCount.name,item_data[Data.applyCount.name])
            self.data.set_data(Data.itemUrl.name,item_data[Data.itemUrl.name])
            self.data.set_data(Data.supportLoans.name,item_data[Data.supportLoans.name])
            self.data.set_data(Data.supportOrgLoan.name,item_data[Data.supportOrgLoan.name])

            self.__do_item_crawling(page,item)

            log(page,item,item_data["status"],start)



    def __do_item_crawling(self,page_id,item_id):

        self.__wait_and_click("CSS_SELECTOR","#pai-item-"+str(self.data.get_data(Data.id.name)),1)

        # Wait for the new window or tab
        self.wait.until(EC.number_of_windows_to_be(2))
        handles = self.driver.window_handles
        self.driver.switch_to.window(handles[1])

       # bid times
        self.wait.until(EC.presence_of_element_located((By.XPATH, "/html[1]/body[1]/div[3]/div[4]/div[1]/div[1]/h1[1]")))
        self.data.set_data(Data.times.name,self.driver.find_element_by_xpath("/html[1]/body[1]/div[3]/div[4]/div[1]/div[1]/h1[1]").get_attribute('textContent'))
        #TODO: degfine new field

       # 变卖公告，竞买公告
        self.__wait_and_click("CSS_SELECTOR","#J_DetailTabMenu > li.current.first > a",0.5)

        self.data.set_data(Data.areaA.name,self.driver.find_element_by_id("J_NoticeDetail").get_attribute('textContent'))


        self.__wait_and_click("LINK_TEXT","标的物介绍",0.5)

        self.data.set_data(Data.areaB.name,self.driver.find_element_by_id("J_desc").get_attribute('textContent'))


        if(self.data.get_data(Data.status.name) == "done"):

            self.__wait_and_click("LINK_TEXT","竞价成功确认书",0.5)

            self.wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrap")))

            name = self.driver.find_element_by_class_name("c-content").text
        elif(self.data.get_data(Data.status.name) == "failure"):
            name = "用户姓名流拍通过"
        else:
            name = "用户姓名即将开始通过"
        self.data.set_data(Data.name.name,name)

        self.data.write_file()
        self.driver.close()
        self.driver.switch_to.window(handles[0])

In [8]:
if __name__ == '__main__':

    start = datetime.datetime.now()
    print("start : {0}".format(start))
    
    sf = SfSpider(G_Debug,G_Debug_Pages,G_Debug_Items,G_Chrome_Hide,sourceUrl,G_FILE_NAME)
    
    sf.do_crawling()
    
    end = datetime.datetime.now()
    print("end : {0}".format(end))
    print("duration: {0}".format(end-start))



start : 2019-08-04 17:24:20.972820
page id: 1, item id: 0, status:done, duration:0:00:04.308249
page id: 1, item id: 1, status:failure, duration:0:00:02.787668
page id: 1, item id: 2, status:done, duration:0:00:03.439928
page id: 1, item id: 3, status:failure, duration:0:00:02.724628
page id: 1, item id: 4, status:failure, duration:0:00:02.681372
page id: 2, item id: 0, status:failure, duration:0:00:02.660663
page id: 2, item id: 1, status:failure, duration:0:00:02.665982
page id: 2, item id: 2, status:failure, duration:0:00:02.632627
page id: 2, item id: 3, status:failure, duration:0:00:02.634514
page id: 2, item id: 4, status:failure, duration:0:00:03.133816
end : 2019-08-04 17:24:57.066185
duration: 0:00:36.093365
